# Washington State DOT Traffic API

source: [https://wsdot.wa.gov/traffic/api](https://wsdot.wa.gov/traffic/api)


In [2]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv
from pathlib import Path
from sqlalchemy import create_engine, text

## Import env variables


In [3]:
# load the .env file
load_dotenv()

# API variables
api_key = os.getenv("API_KEY")

# MySQL database variables
db_user = os.getenv("AZURE_USERNAME")
db_pwd = os.getenv("AZURE_PWD")
db_host = os.getenv("AZURE_URL")
db_port = os.getenv("AZURE_PORT")
db_database = os.getenv("AZURE_DB")

# print(api_key)

In [4]:
    connection_url = (
        f"mysql+pymysql://{db_user}:{db_pwd}" f"@{db_host}:{db_port}/{db_database}"
    )

## Check Table Rows

In [4]:
list_of_tables = [
    "api_fetch",
    "time_travel_raw",
    "traffic_alerts_raw",
    "weather_alerts_raw",
]


def check_table_contents(list_of_tables):
    try:
        # Connect to database
        engine = create_engine(connection_url)
        with engine.connect() as connection:
            for table_name in list_of_tables:
                result = connection.execute(text(f"SELECT COUNT(*) FROM {table_name}"))
                count = result.fetchone()[0]
                print(f"Table {table_name} contains {count} rows.")
    except Exception as e:
        print(f"Database error: {e}")


check_table_contents(list_of_tables)

Table api_fetch contains 1 rows.
Table time_travel_raw contains 168 rows.
Table traffic_alerts_raw contains 122 rows.
Table weather_alerts_raw contains 103 rows.


In [5]:
def display_table_contents(table_name):
    try:
        # Connect to database
        engine = create_engine(connection_url)
        with engine.connect() as connection:
            df = pd.read_sql("SELECT * FROM " + table_name, connection)
            return df
    except Exception as e:
        print(f"Database error: {e}")
        return None

In [6]:
df = display_table_contents("api_fetch")
df.head(10)

,id,travel_times_response,traffic_alerts_response,weather_alerts_response,timestamp
0,1,200,200,200,2025-02-05 21:22:14


In [7]:
df = display_table_contents("api_fetch_hist")
df.head(10)

,id,timestamp,travel_times_response,traffic_alerts_response,weather_alerts_response,status
0,1,2025-02-13 20:38:08,200,200,200,Success
1,2,2025-02-13 20:38:52,200,200,200,Success
2,3,2025-02-13 20:45:45,200,200,200,Success
3,4,2025-02-13 20:56:05,200,200,200,Success
4,5,2025-02-13 20:57:09,200,200,200,Success
5,6,2025-02-13 20:58:59,200,200,200,Success
6,7,2025-02-13 22:17:08,200,200,200,Success
7,8,2025-02-13 22:17:33,200,200,200,Success
8,9,2025-02-13 22:21:39,200,200,200,Success
9,10,2025-02-13 22:22:08,200,200,200,Success


In [7]:
df = display_table_contents("time_travel_raw")
df.head(10)

,id,averagetime,description,currenttime,distance,endpoint,name,startpoint,timeupdated,traveltimeid,timestamp
0,1,25.0,Everett to Downtown Seattle using HOV lanes,25.0,26.72,"{""Latitude"": 47.609294, ""MilePost"": 165.83, ""R...",Everett-Seattle HOV,"{""Latitude"": 47.964146, ""MilePost"": 192.55, ""R...",/Date(1738819200000-0800)/,2,2025-02-05 21:22:14
1,2,25.0,Downtown Seattle to Everett using HOV lanes,25.0,26.94,"{""Latitude"": 47.92428, ""MilePost"": 192.77, ""Ro...",Seattle-Everett HOV,"{""Latitude"": 47.609294, ""MilePost"": 165.83, ""R...",/Date(1738819200000-0800)/,3,2025-02-05 21:22:14
2,3,25.0,Downtown Seattle to Everett,25.0,26.94,"{""Latitude"": 47.92428, ""MilePost"": 192.77, ""Ro...",Seattle-Everett,"{""Latitude"": 47.609294, ""MilePost"": 165.83, ""R...",/Date(1738819200000-0800)/,4,2025-02-05 21:22:14
3,4,9.0,Downtown Bellevue to Issaquah,9.0,9.28,"{""Latitude"": 47.541799, ""MilePost"": 16.96, ""Ro...",Bellevue-Issaquah,"{""Latitude"": 47.6138, ""MilePost"": 13.33, ""Road...",/Date(1738819200000-0800)/,5,2025-02-05 21:22:14
4,5,9.0,Downtown Bellevue to Issaquah using HOV lanes,9.0,9.28,"{""Latitude"": 47.541799, ""MilePost"": 16.96, ""Ro...",Bellevue-Issaquah HOV,"{""Latitude"": 47.6138, ""MilePost"": 13.33, ""Road...",/Date(1738819200000-0800)/,6,2025-02-05 21:22:14
5,6,9.0,Issaquah to Downtown Bellevue using HOV lanes,9.0,9.25,"{""Latitude"": 47.61361, ""MilePost"": 13.6, ""Road...",Issaquah-Bellevue HOV,"{""Latitude"": 47.541799, ""MilePost"": 16.96, ""Ro...",/Date(1738819200000-0800)/,7,2025-02-05 21:22:14
6,7,9.0,Issaquah to Downtown Bellevue,9.0,9.25,"{""Latitude"": 47.61361, ""MilePost"": 13.6, ""Road...",Issaquah-Bellevue,"{""Latitude"": 47.541799, ""MilePost"": 16.96, ""Ro...",/Date(1738819200000-0800)/,8,2025-02-05 21:22:14
7,8,25.0,Downtown Bellevue to Everett,25.0,26.76,"{""Latitude"": 47.92428, ""MilePost"": 192.77, ""Ro...",Bellevue-Everett,"{""Latitude"": 47.6138, ""MilePost"": 13.6, ""RoadN...",/Date(1738819200000-0800)/,9,2025-02-05 21:22:14
8,9,25.0,Everett to Downtown Bellevue,25.0,26.86,"{""Latitude"": 47.617307, ""MilePost"": 13.33, ""Ro...",Everett-Bellevue,"{""Latitude"": 47.964146, ""MilePost"": 192.55, ""R...",/Date(1738819200000-0800)/,10,2025-02-05 21:22:14
9,10,25.0,Downtown Bellevue to Everett using HOV lanes,25.0,26.76,"{""Latitude"": 47.92428, ""MilePost"": 192.77, ""Ro...",Bellevue-Everett HOV,"{""Latitude"": 47.6138, ""MilePost"": 13.6, ""RoadN...",/Date(1738819200000-0800)/,11,2025-02-05 21:22:14


In [8]:
df = display_table_contents("traffic_alerts_raw")
df.head(10)

,id,alertid,county,endroadwaylocation,endtime,eventcategory,eventstatus,extendeddescription,headlinedescription,lastupdatedtime,priority,region,startroadwaylocation,starttime,timestamp
0,1,624112,Grant,"{""Latitude"": 47.569251504, ""MilePost"": 89.56, ...",None,Rest Area,Open,,The blue Lake rest area is closed for the winter.,/Date(1731629066113-0800)/,Low,North Central,"{""Latitude"": 47.569251504, ""MilePost"": 89.56, ...",/Date(1731628800000-0800)/,2025-02-05 21:22:14
1,2,626164,Spokane,"{""Latitude"": 47.587515659, ""MilePost"": 91.18, ...",None,Road Report,Open,,"US195 from Rosalia to Spokane, road conditions...",/Date(1738815414627-0800)/,Medium,Eastern,"{""Latitude"": 47.253450503, ""MilePost"": 65.78, ...",/Date(1738816200000-0800)/,2025-02-05 21:22:14
2,3,632835,None,"{""Latitude"": 47.504307562, ""MilePost"": 27.59, ...",None,Construction,Open,,Eastbound and westbound SR 18 will shift to th...,/Date(1737761204897-0800)/,Low,Northwest,"{""Latitude"": 47.486733564, ""MilePost"": 26.36, ...",/Date(1738155600000-0800)/,2025-02-05 21:22:14
3,4,561051,None,"{""Latitude"": 46.986509592, ""MilePost"": 13.3, ""...",/Date(1767196800000-0800)/,Construction,Open,,US 12 is reduced to one lane in each direction...,/Date(1733323301377-0800)/,Medium,Olympic,"{""Latitude"": 46.982067911, ""MilePost"": 11.95, ...",/Date(1691532000000-0700)/,2025-02-05 21:22:14
4,5,634451,None,"{""Latitude"": 46.762821143, ""MilePost"": 16.8, ""...",/Date(1738888200000-0800)/,Maintenance,Open,,"Monday, Feb. 3 to Thursday, Feb. 6: Travelers ...",/Date(1738364947003-0800)/,Medium,Southwest,"{""Latitude"": 46.559042394, ""MilePost"": 0.5, ""R...",/Date(1738587600000-0800)/,2025-02-05 21:22:14
5,6,583671,None,"{""Latitude"": 47.579650228, ""MilePost"": 11.03, ...",/Date(1771765200000-0800)/,Construction,Open,,The westbound I-90 HOV lane will close between...,/Date(1708723313933-0800)/,Medium,Northwest,"{""Latitude"": 47.580257883, ""MilePost"": 9.72, ""...",/Date(1708723080000-0800)/,2025-02-05 21:22:14
6,7,626158,Lincoln,"{""Latitude"": 47.433126691, ""MilePost"": 257.42,...",None,Road Report,Open,,"I-90 from Ritzville to SR 904, road conditions...",/Date(1738808964377-0800)/,Low,Eastern,"{""Latitude"": 47.111077833, ""MilePost"": 220.1, ...",/Date(1738809300000-0800)/,2025-02-05 21:22:14
7,8,468632,None,"{""Latitude"": 47.6390387, ""MilePost"": 5.0, ""Roa...",None,Flammable Cargo Restriction,Open,,Flammable cargo is prohibited on SR 520 eastbo...,/Date(1631541620707-0700)/,High,Northwest,"{""Latitude"": 47.644373883, ""MilePost"": 1.0, ""R...",/Date(1631541540000-0700)/,2025-02-05 21:22:14
8,9,619219,None,"{""Latitude"": 47.678713174, ""MilePost"": 80.0, ""...",None,Construction,Open,,Upriver Drive will have intermittent single la...,/Date(1738250900920-0800)/,High,Eastern,"{""Latitude"": 47.677765521, ""MilePost"": 90.0, ""...",/Date(1728660660000-0700)/,2025-02-05 21:22:14
9,10,626163,Whitman,"{""Latitude"": 47.253450503, ""MilePost"": 65.78, ...",None,Road Report,Open,,"US195 from Pullman to Rosalia, road conditions...",/Date(1738808710420-0800)/,Low,Eastern,"{""Latitude"": 46.738118033, ""MilePost"": 22.53, ...",/Date(1738809000000-0800)/,2025-02-05 21:22:14


In [6]:
df = display_table_contents("weather_alerts_raw")
df.head(10)

,BarometricPressure,Latitude,Longitude,PrecipitationInInches,ReadingTime,RelativeHumidity,SkyCoverage,StationID,StationName,TemperatureInFahrenheit,Visibility,WindDirection,WindDirectionCardinal,WindGustSpeedInMPH,WindSpeedInMPH,timestamp
0,960.9,47.474800,-122.270400,None,/Date(1739893507000-0800)/,99.0,N/A,1909,S 144th St on SB I-5 at mp 155.32,44.96,1.0,120.0,ESE,1.0,0.0,2025-02-18 08:00:03
1,NaN,47.760633,-122.184048,None,/Date(1739893517000-0800)/,100.0,N/A,1910,NE 195th on SB I-405 at mp 24.58,43.34,1.0,5.0,N,0.0,0.0,2025-02-18 08:00:03
2,985.4,47.509000,-121.885000,None,/Date(1739893508000-0800)/,99.0,N/A,1928,EB I-90 / SR-18 (Echo Lake) at mp 26.30,39.74,1.0,247.0,WSW,3.0,0.0,2025-02-18 08:00:03
3,1006.8,47.726000,-122.324000,None,/Date(1739893504000-0800)/,98.0,N/A,1966,NE 130th Street on I-5 at mp 173.75,43.88,1.0,NaN,N/A,3.0,0.0,2025-02-18 08:00:03
4,NaN,46.436000,-117.350000,None,/Date(1739893953000-0800)/,97.0,N/A,1968,Alpowa Summit on US 12 at mp 413.36,29.12,0.0,162.0,SSE,2.0,2.0,2025-02-18 08:00:03
5,NaN,47.033800,-123.097100,None,/Date(1739893895000-0800)/,99.0,N/A,1970,Rock Candy Mountain on SR 8 at mp 16.11,42.44,1.0,NaN,N/A,2.0,0.0,2025-02-18 08:00:03
6,NaN,47.555500,-120.586600,None,/Date(1739893814000-0800)/,84.0,N/A,1972,Dryden Road on US 2 at mp 104.81,29.84,0.0,130.0,SE,1.0,0.0,2025-02-18 08:00:03
7,NaN,48.070300,-124.049000,None,/Date(1739893981000-0800)/,100.0,N/A,1975,Heckelsville Shed on US 101 at mp 214.75,33.98,1.0,0.0,N/A,2.0,0.0,2025-02-18 08:00:03
8,NaN,46.887000,-120.427000,None,/Date(1739893545000-0800)/,NaN,N/A,1977,Manastash Ridge Eastbound on I-82 at mp 7.76,28.04,1.0,NaN,N/A,2.0,0.0,2025-02-18 08:00:03
9,NaN,46.928000,-120.500000,None,/Date(1739893551000-0800)/,98.0,N/A,1978,Manastash Ridge Westbound on I-82 at mp 3.75,28.04,1.0,175.0,S,7.0,6.0,2025-02-18 08:00:03
